# Week 11, Prep notebook

Last week we talked about how Jekyll sites work overall, and this week we'll focus on building visualizations to host on these pages with vega-lite and Altair in Python.

## 1. Include vega-lite plots directly from vega-editor

We can start with saving vega-lite code directly as json if we do our development in the vega-editor.  For a walk through of this see the file `3_direct_from_vega_editor.md` in the prep files (note to self: this is in the dev onlinecv locally at the moment).

**You must include the full URL to the dataset for this to work!**

## 2. Copy vega-lite code from other sources and save using Altair

For example, you may have developed some code on [Starboard](https://starboard.gg/) that you now want to copy into this notebook with Altair, and then eventually save to your Jekyll webpage.

We can follow the [Converting vega-lite to Altair instructions](https://altair-viz.github.io/user_guide/internals.html#converting-vega-lite-to-altair) to do this.

For example, from [one of our prep Starboard notebooks](https://starboard.gg/jnaiman/prep_notebook_week11_spring2022-nG3SEUx) we have the following vega-lite specification:

```javascript
var myHist1 = 
{
  data: {"url": "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"
  },
  mark: "bar",
  height: "300",
  width: "500",
  encoding: {
    "x": {"field": "State", "type": "nominal"},
    // NOTE: this won't work because "sum" assumes numerical data
    //"y": {"aggregate": "sum", "field": "State", "type": "nominal"} 
    //"y": {"aggregate": "count", "field": "State", "type": "nominal"} // might give an error/warning
    "y": {"aggregate": "count", "field": "State", "type": "quantitative"}
  }
};

var v = vegaEmbed('#firstHist', myHist1);
```

We can convert this to an Altair plot here with:

In [3]:
import altair as alt

In [100]:
chart1 = alt.Chart.from_dict({
  "data": {"url": "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"
  },
  "mark": "bar",
  "height": 300,
  "width": 500,
  "encoding": {
    "x": {"field": "State", "type": "nominal"},
    # NOTE: this won't work because "sum" assumes numerical data
    #"y": {"aggregate": "sum", "field": "State", "type": "nominal"} 
    #"y": {"aggregate": "count", "field": "State", "type": "nominal"} // might give an error/warning
    "y": {"aggregate": "count", "field": "State", "type": "quantitative"}
  }
})

chart1

alt.Chart(...)

Note that I had to put in and take out some " marks -- this is because different vega-lite "hosting" services will be more/less lenient with these kinds of formatting issues.

Now, in principle, I could go to the Vega-Editor using the three `...` button at the upper right of this plot, OR I can just save with:

In [6]:
myJekyllDir = '/Users/jnaiman/online_cv_fall2022/'

In [101]:
chart1.properties(width='container').save(myJekyllDir+"assets/json/chart1.json")

## 3. Dashboard Building

So, not everything translates easily between how we've used vega-lite in Starboard before to here, in Jekyll.  In particular, selections in between plots can get a little tricky.  To do this, let's see about re-making our Mobility dashboard. 

Our original dashboard code from [our prior Starboard notebook](https://starboard.gg/jnaiman/inClass_week12_spring2022-nZa79A6) looks like:

```javascript
var connectedSpec = {
  // Data -- same for both plots, so we just do one call
  data: {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"},
  hconcat:[
    // ------------------ SPECS FOR PLOT 1 - rect plot --------------
    { // start, plot 1
        // ADD: a selection on the rectangle plot
        "params": [{"name": "pts", "select": "interval"}],	    
      // Marks
  		mark:"rect",
  		height:"400",
  		// Encoding (note:error for encoding vs encodings)
  		encoding:{
    		"x":{"bin":{"maxbins":10}, "field":"Student_teacher_ratio", "type":"quantitative"},
    		"y":{"field":"State","type":"ordinal"},
    		"color":{"aggregate":"count", "type":"quantitative"} 
    		// will show the number of records with a specific student/teacher ratio in a particular state
        } // make sure close encoding here!
    }, // end plot 1
    { // start plot 2
         // ADD: a transformation to filter our data based on this selection
         transform:[{"filter":{"param":"pts"}}], // should be "param" not "params"
  		// Mark
  		mark: "bar",
        height:"400",
  		// Encoding
  		encoding:{
    		"x":{"field":"Mobility", "type":"quantitative", "bin":true, "axis":{"title":"Mobility Score"}},
    		"y":{"aggregate":"count","type":"quantitative", "axis":{"title":"Mobility Score Distribution"}}
  		}      
    } // end plot 2
    // ------------------ SPECS FOR PLOT 2 - histogram ---------------
  ] // end of hconcat  
};

var v = vegaEmbed("#connectedSidebyside",connectedSpec);
```

Translating this to a `from_dict` call in Altair we get:

In [102]:
chart = alt.Chart.from_dict({
  #// Data -- same for both plots, so we just do one call
  "data": {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"},
  "hconcat":[
    #// ------------------ SPECS FOR PLOT 1 - rect plot --------------
    { #// start, plot 1
        #// ADD: a selection on the rectangle plot
        "params": [{"name": "pts", "select": "interval"}],	    
      #// Marks
  		"mark":"rect",
  		"height":400,
  		#// Encoding (note:error for encoding vs encodings)
  		"encoding":{
    		"x":{"bin":{"maxbins":10}, "field":"Student_teacher_ratio", "type":"quantitative"},
    		"y":{"field":"State","type":"ordinal"},
    		"color":{"aggregate":"count", "type":"quantitative"} 
    		#// will show the number of records with a specific student/teacher ratio in a particular state
        } #// make sure close encoding here!
    }, #// end plot 1
    { #// start plot 2
         #// ADD: a transformation to filter our data based on this selection
         "transform":[{"filter":{"param":"pts"}}], #// should be "param" not "params"
  		#// Mark
  		"mark": "bar",
        "height":400,
  		#// Encoding
  		"encoding":{
    		"x":{"field":"Mobility", "type":"quantitative", "bin":True, "axis":{"title":"Mobility Score"}},
    		"y":{"aggregate":"count","type":"quantitative", "axis":{"title":"Mobility Score Distribution"}}
  		}      
    } #// end plot 2
    #// ------------------ SPECS FOR PLOT 2 - histogram ---------------
  ] #// end of hconcat  
})
chart

ValidationError: {'data': {'url': 'https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv'}, 'hconcat': [{'params': [{'name': 'pts', 'select': 'interval'}], 'mark': 'rect', 'height': 400, 'encoding': {'x': {'bin': {'maxbins': 10}, 'field': 'Student_teacher_ratio', 'type': 'quantitative'}, 'y': {'field': 'State', 'type': 'ordinal'}, 'color': {'aggregate': 'count', 'type': 'quantitative'}}}, {'transform': [{'filter': {'param': 'pts'}}], 'mark': 'bar', 'height': 400, 'encoding': {'x': {'field': 'Mobility', 'type': 'quantitative', 'bin': True, 'axis': {'title': 'Mobility Score'}}, 'y': {'aggregate': 'count', 'type': 'quantitative', 'axis': {'title': 'Mobility Score Distribution'}}}}]} is not valid under any of the given schemas

Failed validating 'anyOf' in schema:
    {'anyOf': [{'$ref': '#/definitions/TopLevelUnitSpec'},
               {'$ref': '#/definitions/TopLevelFacetSpec'},
               {'$ref': '#/definitions/TopLevelLayerSpec'},
    

So, we run into a couple of errors, with the (*waves hands*) issue being related to `params` in Altair and concatination support. So, let's try a different strategy -- making each plot again and using Altair to do the concatination for us.

Let's first do the plots one at a time.  The rectangle plot spec looked like:

```javascript
var rectPlot1Spec = {
  // Data
  data: {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"},
  // Marks
  mark:"rect",
  height:"400",
  // Encoding (note:error for encoding vs encodings)
  encoding:{
    //"x":{"field":"Student_teacher_ratio", "type":"quantitative"},
    "x":{"bin":{"maxbins":10}, "field":"Student_teacher_ratio", "type":"quantitative"},
    "y":{"field":"State","type":"ordinal"},
    "color":{"aggregate":"count", "type":"quantitative"} 
    // will show the number of records with a specific student/teacher ratio in a particular state
  }
  
};

var v = vegaEmbed('#rectPlot1',rectPlot1Spec);
```

Which is translated to:

In [103]:
chart1 = alt.Chart.from_dict({
  #// Data
  "data": {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"},
  #// Marks
  "mark":"rect",
  "height":400,
  #// Encoding (note:error for encoding vs encodings)
  "encoding":{
    #//"x":{"field":"Student_teacher_ratio", "type":"quantitative"},
    "x":{"bin":{"maxbins":10}, "field":"Student_teacher_ratio", "type":"quantitative"},
    "y":{"field":"State","type":"ordinal"},
    "color":{"aggregate":"count", "type":"quantitative"} 
    #// will show the number of records with a specific student/teacher ratio in a particular state
  }
  
})
chart1

alt.Chart(...)

Note of course the changes in where " have been added and removed.

The second chart was the histogram of the mobility score, given by the specification:

```javascript
var mobilityHistSpec = {
  // Data
  data: {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"},
  // Mark
  mark: "bar",
  // Encoding
  encoding:{
    "x":{"field":"Mobility", "type":"quantitative", "bin":true, "axis":{"title":"Mobility Score"}},
    //"x":{"field":"Mobility", "type":"quantitative", "axis":{"title":"Mobility Score"}},
    "y":{"aggregate":"count","type":"quantitative", "axis":{"title":"Mobility Score Distribution"}}
  }
};

var v = vegaEmbed('#mobilityHist1',mobilityHistSpec);
```

This now turns into:

In [104]:
chart2 = alt.Chart.from_dict({
  #// Data
  "data": {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"},
  #// Mark
  "mark": "bar",
  #// Encoding
  "encoding":{
    "x":{"field":"Mobility", "type":"quantitative", "bin":True, "axis":{"title":"Mobility Score"}},
    #//"x":{"field":"Mobility", "type":"quantitative", "axis":{"title":"Mobility Score"}},
    "y":{"aggregate":"count","type":"quantitative", "axis":{"title":"Mobility Score Distribution"}}
  }
})
chart2

alt.Chart(...)

Note here that we had to change the JS `true` to the Pythonic `True`.

The first thing we probably want to do is put these charts side-by-side.  We can do this with Altair's horizontal concatination function:

In [105]:
chart = alt.HConcatChart(hconcat=[chart1,chart2])
chart

alt.HConcatChart(...)

In [106]:
# note that this gives an error -- ignore this for the time being
#chart.properties(width='container').save(myJekyllDir+"assets/json/static_mobility.json")
chart.save(myJekyllDir+"assets/json/static_mobility.json")

Note that the "width" parameter only [refers to sub-charts for faceted charts](https://altair-viz.github.io/user_guide/customization.html#adjusting-chart-size) so we are stuck adjusting this on our own if we want to change the height of these plots.  This means these facet charts won't have "responsive" sizes, which is a bit of a bummer, but will hopefully be something they update in the future!

Ok, so now we want to add a brush selection like before for the left-most plot that changes the values of the histograms on the right-most plot.  How to do this?  Well, first let's look at the brush selector in Altair:

In [107]:
brush = alt.selection_interval()  # selection of type "interval"

We can add this to our first plot and then see we have some interactivity:

In [108]:
chart1.add_selection(
        brush
    )

alt.Chart(...)

What are some of the parameters we can use in our brush selection?

In [109]:
alt.selection_interval?

One thing we note here is that there is potentially a `field` we can use for the selection which might make us think we want to use "Mobility" as the input but we actually now need to specify the encodings -- i.e. on what axis in our original plot are we selecting?

So, we are selecting boxes in `chart1` -- so this will be selecting on both x & y:

In [120]:
brush = alt.selection_interval(encodings=['x','y'])

Let's now add this brush to our `chart1`:

In [121]:
chart1 = alt.Chart.from_dict({
  "data": {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"},
  "mark":"rect",
  "height":400,
  "encoding":{
    "x":{"bin":{"maxbins":10}, "field":"Student_teacher_ratio", "type":"quantitative"},
    "y":{"field":"State","type":"ordinal"},
    "color":{"aggregate":"count", "type":"quantitative"} 
  }  
}).add_selection(
        brush
    )

And now we can add this also as a [transform filter](https://altair-viz.github.io/user_guide/transform/filter.html) to our `chart2` plot:

In [122]:
chart2 = alt.Chart.from_dict({
  "data": {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"},
  "mark": "bar",
  "encoding":{
    "x":{"field":"Mobility", "type":"quantitative", "bin":True, "axis":{"title":"Mobility Score"}},
    "y":{"aggregate":"count","type":"quantitative", "axis":{"title":"Mobility Score Distribution"}}
  }
}).transform_filter(
    brush
)

Now let's horizontally concatinate them, and let's do it in the fancy Altair way:

In [130]:
chart = chart1 | chart2

In [131]:
chart

alt.HConcatChart(...)

Neat!  Now we can once again save this in the "usual way" to add to our Jekyll file:

In [54]:
#error again, ignorning
#chart.properties(width='container').save(myJekyllDir+"assets/json/dashboard_mobility.json")
chart.save(myJekyllDir+"assets/json/dashboard_mobility.json")

## 4. Use Altair to make the chart 

To build straight from Python, we need to read in the data first.  Before, we've been linking to online data, but the nice thing about Altair is that instead of doing data manipulations "on the fly" in vega-lite, we can potentially do them in Python and then save the modified data for our plot.

Before doing that though, let's see if we can translating from vega-lite style to "Altair-vega-lite style".  We can do this with our data still stored online at a url:

In [1]:
mobility_url = 'https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv'

Let's once again re-write `chart1`, but now transforming from the dictionary we have been passing to more "original Altair" formatting and passing data through Python:

Before doing any complicated binning stuff, let's start with a simple scatter plot in "Altair style":

In [79]:
scatter1 = alt.Chart(mobility_url).mark_point().encode(
    x='Mobility:Q', # "Q for quantiative"
    y='Population:Q',
)
scatter1

alt.Chart(...)

Note here we used `Q` for quantitative which is one of the [Altair Encoding Data Types](https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types):

| Data Type | Shorthand Code | Description |
|-----------|----------------|-------------|
| quantitative | Q | a continuous real-valued quantity |
| ordinal | O | a discrete ordered quantity |
| nominal | N | a discrete unordered category |
| temporal | T | a time or date value |
| geojson | G | a geographic shape |

Also note that now we define what mark we are using, in this case `mark_point` after we define the "source" of the chart (in this case a URL).

Let's make this scatter plot a bit more complex by coloring by another quantitative variable, like `Income`:

In [80]:
scatter2 = alt.Chart(mobility_url).mark_point().encode(
    x='Mobility:Q', # "Q for quantiative"
    y='Population:Q',
    color=alt.Color('Income:Q')
)
scatter2

alt.Chart(...)

We can pick different color schemes in the same way that we use [Vega-lite colormaps](https://vega.github.io/vega/docs/schemes/#reference) and by specifying the scale for the color in Altair:

In [81]:
scatter3 = alt.Chart(mobility_url).mark_point().encode(
    x='Mobility:Q', # "Q for quantiative"
    y='Population:Q',
    color=alt.Color('Income:Q', scale=alt.Scale(scheme='viridis'))
)
scatter3

alt.Chart(...)

So, this is a bit more readable, however we still need to do a few things.  To practice binning, let's [bin our color](https://altair-viz.github.io/user_guide/transform/bin.html#bin-transforms) from our `Income` variable on the color bar and in our plot:

In [83]:
scatter4 = alt.Chart(mobility_url).mark_point().encode(
    x='Mobility:Q', # "Q for quantiative"
    y='Population:Q',
    color=alt.Color('Income:Q', scale=alt.Scale(scheme='viridis'),bin=alt.Bin(maxbins=5))
)
scatter4

alt.Chart(...)

For this, if we really want to highlight the bins, we probably want more hues, so let's change our colormap:

In [84]:
scatter5 = alt.Chart(mobility_url).mark_point().encode(
    x='Mobility:Q', # "Q for quantiative"
    y='Population:Q',
    color=alt.Color('Income:Q', scale=alt.Scale(scheme='sinebow'),bin=alt.Bin(maxbins=5))
)
scatter5

alt.Chart(...)

Groovy!  Also, it does look like we have a pretty big range in the `Population` parameter so maybe we want a log scale on our y-axis:

In [9]:
scatter6 = alt.Chart(mobility_url).mark_point().encode(
    x='Mobility:Q', # "Q for quantiative"
    #y='Population:Q',
    y=alt.Y('Population:Q', scale=alt.Scale(type='log')),
    color=alt.Color('Income:Q', scale=alt.Scale(scheme='sinebow'),bin=alt.Bin(maxbins=5))
)
scatter6

alt.Chart(...)

Nice!  Let's save that one:

In [10]:
scatter6.save(myJekyllDir+"assets/json/population_scatter.json")

#### ASIDE

Now, let's try to save this with a smaller width.  One temptation would be to just add the width property before saving:

In [11]:
scatter7 = alt.Chart(mobility_url).mark_point().encode(
    x='Mobility:Q', # "Q for quantiative"
    #y='Population:Q',
    y=alt.Y('Population:Q', scale=alt.Scale(type='log')),
    color=alt.Color('Income:Q', scale=alt.Scale(scheme='sinebow'),bin=alt.Bin(maxbins=5))
)
scatter7

alt.Chart(...)

In [12]:
scatter7.properties(width='container').save(myJekyllDir+"assets/json/population_scatter.json")

#### END ASIDE

Now that we know a bit more about Altair-flavored vega-lite plots, let's try to remake our dashboard plot, but using only Altair-style.

Let's start with the first plot.

Our `from_dict` call looks like:

```javascript
chart1 = alt.Chart.from_dict({
  "data": {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"},
  "mark":"rect",
  "height":400,
  "encoding":{
    "x":{"bin":{"maxbins":10}, "field":"Student_teacher_ratio", "type":"quantitative"},
    "y":{"field":"State","type":"ordinal"},
    "color":{"aggregate":"count", "type":"quantitative"} 
  }  
})
```

We can use the [`mark_rect` Altair encoding example](https://altair-viz.github.io/gallery/interactive_cross_highlight.html#interactive-chart-with-cross-highlight) to build this plot:

In [15]:
chart1 = alt.Chart(mobility_url).mark_rect().encode(
    alt.X("Student_teacher_ratio:Q", bin=alt.Bin(maxbins=10)),
    alt.Y("State:O"),
    alt.Color("count()")
).properties(
   height=400
)
chart1

alt.Chart(...)

Neat!  Let's also re-make our second chart.  For reference we had the following `from_dict` call:

```javascript
chart2 = alt.Chart.from_dict({
  "data": {"url":"https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv"},
  "mark": "bar",
  "encoding":{
    "x":{"field":"Mobility", "type":"quantitative", "bin":True, "axis":{"title":"Mobility Score"}},
    "y":{"aggregate":"count","type":"quantitative", "axis":{"title":"Mobility Score Distribution"}}
  }
})
```

We can [bin using `mark_bar`](https://altair-viz.github.io/user_guide/transform/bin.html#bin-transforms).

In [22]:
chart2 = alt.Chart(mobility_url).mark_bar().encode(
    alt.X("Mobility:Q", bin=True,axis=alt.Axis(title='Mobility Score')),
    alt.Y('count()', axis=alt.Axis(title='Mobility Score Distribution'))
)
chart2

alt.Chart(...)

We can then put them side by side once more:

In [33]:
chart = (chart1.properties(width=300) | chart2.properties(width=300))

In [34]:
chart

alt.HConcatChart(...)

Now, we can essentially use the code we used before to make these charts interactive with each other:

In [36]:
brush = alt.selection_interval(encodings=['x','y'])

chart1 = alt.Chart(mobility_url).mark_rect().encode(
    alt.X("Student_teacher_ratio:Q", bin=alt.Bin(maxbins=10)),
    alt.Y("State:O"),
    alt.Color("count()")
).properties(
   height=400
).add_selection(
        brush
)

chart2 = alt.Chart(mobility_url).mark_bar().encode(
    alt.X("Mobility:Q", bin=True,axis=alt.Axis(title='Mobility Score')),
    alt.Y('count()', axis=alt.Axis(title='Mobility Score Distribution'))
).transform_filter(
    brush
)

chart = (chart1.properties(width=300) | chart2.properties(width=300))

chart

alt.HConcatChart(...)

Great!  Let's save that again:

In [37]:
chart.save(myJekyllDir+"assets/json/altair_mobility_dashboard.json")

## 5. Python Analysis + Altair Plotting

We can also do all of the data transformations in Python (which might be easier since we've been using Python this whole time!) and then do the actual plotting in Altair... this won't work for re-doing the above plot since the binning of the data happens "on the fly" BUT I bet we could re-do this as a pure rectangle plot and then allow for the selection of individual state+mobility score combos...

In [65]:
import pandas as pd

In [73]:
mobility = pd.read_csv('https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/mobility.csv')

In [74]:
mobility.head()

,ID,Name,Mobility,State,Population,Urban,Black,Seg_racial,Seg_income,Seg_poverty,...,Migration_out,Foreign_born,Social_capital,Religious,Violent_crime,Single_mothers,Divorced,Married,Longitude,Latitude
0,100,Johnson City,0.062199,TN,576081,1,0.021,0.090,0.035,0.030,...,0.005,0.012,-0.298,0.514,0.001,0.190,0.110,0.601,-82.436386,36.470371
1,200,Morristown,0.053652,TN,227816,1,0.020,0.093,0.026,0.028,...,0.014,0.023,-0.767,0.544,0.002,0.185,0.116,0.613,-83.407249,36.096539
2,301,Middlesborough,0.072635,TN,66708,0,0.015,0.064,0.024,0.015,...,0.012,0.007,-1.270,0.668,0.001,0.211,0.113,0.590,-83.535332,36.551540
3,302,Knoxville,0.056281,TN,727600,1,0.056,0.210,0.092,0.084,...,0.014,0.020,-0.222,0.602,0.001,0.206,0.114,0.575,-84.242790,35.952259
4,401,Winston-Salem,0.044801,NC,493180,1,0.174,0.262,0.072,0.061,...,0.019,0.053,-0.018,0.488,0.003,0.220,0.092,0.586,-80.505333,36.081276
